# Dealing with imbalanced data <img src="logo.png",width=140,height=140, align="right">

This notebook shows you a few tricks how to tackle imbalanced data. We'll use a libray called imblearn. You can find more information about how to use this library here:

http://contrib.scikit-learn.org/imbalanced-learn/

It's a very useful website where you can find lots of hands on examples. 

## Installing a new library

Firstly, before we can start, we need to install a new library for dealing with imbalanced data. The steps how to so this are as follows:

1. Open a terminal window in your workspace
2. write "pip install imblearn"
3. Done, you should be ready to run this notebook

Alternatively, you can stay in your notebook and type:

In [ ]:
!pip install imblearn

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.plotly as py
import pylab
import scipy

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn import feature_selection
from sklearn import preprocessing
from sklearn import neighbors as neigh
import seaborn as sns
from sklearn.metrics import classification_report

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn import pipeline as pl
from imblearn.metrics import classification_report_imbalanced

plt.style.use('seaborn-white')
%matplotlib inline

Let's start by uploading some data. The data we're going to use is actually "synthetic" data, i.e. is generated from random sample method. Actual data will never behave as nice as created, so keep that in the back of your mind when working on this. In order to stay in our wine based theme, we've named the data wine_date_1, 2 and 3 etc, so let's pretend we're going to classify wines based on Alcohol content and acidity only. Start by uploading 3 different datasets: 

In [ ]:
df1=pd.read_csv('wine_data_1.csv')
df2=pd.read_csv('wine_data_2.csv')
df3=pd.read_csv('wine_data_3.csv')
df1.head()

For some reason this dataset has a weird column, so we'll need to do some cleaning and prepping in order to used it in a machine learning algorithm. We don't have to feature scale this data, as it is already on a nice "normal distribiution" scale. Let's define a simple function that preps the data into a X array and y array:

In [ ]:
def prep_data(df):
    X = df.iloc[:, 2:4]
    X = np.array(X).astype(np.float)
    y = df.iloc[:, 1]
    y=np.array(y).astype(np.float)
    return X,y

OK, we'll use this nifty little function over and over to call the dataset that we need. No need to worry about the details for this. Let's have a first look at our datasets. We start by plotting dataset number 1. 

In [ ]:
X,y=prep_data(df3)

plt.scatter(X[y == 0, 0], X[y == 0, 1], label="Class #0", alpha=0.5, linewidth=0.15)
plt.scatter(X[y == 1, 0], X[y == 1, 1], label="Class #1", alpha=0.5, linewidth=0.15)

plt.legend()
plt.show()

**Exercise:** Change the input for the "prep_data" function to df2 and then to df3. Check how the balance between classes changes on the plot. 

Let's have a closer look at the precise numbers for those classes. I'm going to plot three barcharts of each dataset, that should give as a better idea of the ratio and the classes in each dataset

In [ ]:
for df in (df1, df2, df3):
    wineorigin=pd.DataFrame(df['Wine type (Class)'].value_counts())
    plt.figure()
    wineorigin.plot(kind='bar')
    plt.xlabel('Class')
    plt.ylabel('Number of instances')


Alright, that makes it pretty clear doesn't it. Do you suspect any problems with these distributions? Now let's run a very simple K-NN algorithm to predict classes. This should look familiar by now. We'll start by using the data of df1. 

In [ ]:
X,y=prep_data(df1)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.7, random_state=0)

K=1
clf = neigh.KNeighborsClassifier(K)
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)

accuracy = metrics.accuracy_score(y_test, predictions)*100
print('Accuracy: ' + repr(accuracy) + '%')
print (classification_report(y_test, predictions))

Not bad, it seems that it works reasonably well on the first dataset, not perfectly though, but don't worry about this for the moment. 

**Exercise:** Let's repeat running this K-NN algorithm on df2 and df3. Do you notice where things go wrong with the evaluation metrics? 

So do you think df2 has a problem? And what about df3. We'll I think df3 looks like a typical imbalanced classes problem, so let's do something about it. Let's try and resample some of the data in df3. 

## Random Oversampling 

This method creates more data from taking random draws from the minority class, let's see what this looks like in code. Don't worry about the specific code here, it's more important that you understand the implications of over-sampling. You should be able to replicate this code should you ever want to use random oversampling. 

For further reading, I can also recommend looking at this paper, it's a nice explanation of a few different ways to deal with imbalanced data: http://storm.cis.fordham.edu/gweiss/papers/dmin07-weiss.pdf


In [ ]:
# Start by feeding in the X and y we want to use for this exercise. 
X,y=prep_data(df3)

# THIS IS WHERE THE MAGIC HAPPENS
# we'll apply a thing called randomoversampler on X and Y
ros = RandomOverSampler()
X_resampled, y_resampled = ros.fit_sample(X, y)

OK, so now we have normal X and y, as well as X_resampled and y_resamples, the latter is the oversampled data. 
We can put the two datasets in a similar plot to the one above to see the effect of what just happened.

In [ ]:
# Start a plot figure
f, (ax1, ax2) = plt.subplots(1, 2)

# sub-plot number 1, this is our normal data
c0 = ax1.scatter(X[y == 0, 0], X[y == 0, 1], label="Class #0",alpha=0.5)
c1 = ax1.scatter(X[y == 1, 0], X[y == 1, 1], label="Class #1",alpha=0.5)
ax1.set_title('Original set')

# sub-plot number 2, this is our oversampled data
ax2.scatter(X_resampled[y_resampled == 0, 0], X_resampled[y_resampled == 0, 1], label="Class #0", alpha=.5)
ax2.scatter(X_resampled[y_resampled == 1, 0], X_resampled[y_resampled == 1, 1], label="Class #1", alpha=.5)
ax2.set_title('Random over-sampling')

# some settings and ready to go
plt.figlegend((c0, c1), ('Class #0', 'Class #1'), loc='lower center',
              ncol=2, labelspacing=0.)
plt.tight_layout(pad=3)
plt.show()

Right, so how do we interpret this plot? In the right plot the green dots (minority class) just seem thicker. 

**Exercise:** What do you think the thicker green dots represent? 

OK, now let's run a KNN algorithm on the oversampled data, that allows us to see what happens to model performance....

In [ ]:
# feeding in the resampled data into our KNN algo
X_train, X_test, y_train, y_test = train_test_split(X_resampled,y_resampled, test_size=0.7, random_state=0)

K=1
clf = neigh.KNeighborsClassifier(K)
clf.fit(X_train, y_train)

predictions1 = clf.predict(X_test)

accuracy = metrics.accuracy_score(y_test, predictions1)*100
print('Accuracy: ' + repr(accuracy) + '%')
print (classification_report(y_test, predictions1))

OK, that almost looks like it's perfect now! That's slightly weird, perhaps I'm a very skeptical data scientist but results that are too perfect are very suspicious to me. 

** Exercise:** Why does over-sampling in our case lead to almost perfect prediction? 

## Synthetic Minority Oversampling Technique (SMOTE)

SMOTE is considered a better way of increasing the number of minority cases than simply duplicating existing cases like random oversampling does.

SMOTE works by generating new instances from existing cases that you supply as input. The new instances are created by taking samples of the feature space for each target class and its nearest neighbors (so like K-NN), and generating new examples that combine features of the target case with features of its neighbors. This approach increases the features available to each class and makes the samples more general.

There are different ways to execute SMOTE, I won't go into detail here of the different methods, but have a look at this paper that explains a few methods: http://www.ijetae.com/files/Volume2Issue4/IJETAE_0412_07.pdf

Now let's have a look at how this is done in code:

In [ ]:
# Ignore this little function, this is just to help us plot stuff in the loop
def plot_resampling(ax, X, y, title):
    c0 = ax.scatter(X[y == 0, 0], X[y == 0, 1], label="Class #0", alpha=0.5)
    c1 = ax.scatter(X[y == 1, 0], X[y == 1, 1], label="Class #1", alpha=0.5)
    ax.set_title(title)
    return c0, c1


# THIS IS WHERE THE MAGIC HAPPENS!
# We'll take 4 methods of SMOTE and create 4 different datasets in this little loop
kind = ['regular', 'borderline1', 'borderline2', 'svm']
sm = [SMOTE(kind=k) for k in kind]
X_resampled = []
y_resampled = []
for method in sm:
    X_res, y_res = method.fit_sample(X, y)
    X_resampled.append(X_res)
    y_resampled.append(y_res)


# Set the layout for a few subplots
f, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2)
# Remove axis for second plot (this will be empty space, you'll see)
ax2.axis('off')
ax_res = [ax3, ax4, ax5, ax6]

# Boom, now we're using that little function above to create our subplots in a loop
c0, c1 = plot_resampling(ax1, X, y, 'Original set')
for i in range(len(kind)):
    plot_resampling(ax_res[i], X_resampled[i], y_resampled[i],
                    'SMOTE {}'.format(kind[i]))

# do some prettyfying of labels etc
ax2.legend((c0, c1), ('Class #0', 'Class #1'), loc='center',
           ncol=1, labelspacing=0.)
plt.tight_layout()
plt.show()

Don't worry if you don't understand the exact code to create this. It's mostly to show you that indeed, the **different SMOTE methods lead to different data**, so that you should be aware of this when using SMOTE. As you can imagine, the chose method will definitely impact your final results, so when being robust you might want to try a few different methods. 

Right, so let's pick one SMOTE method to resample our data and re-run our KNN algorithm, let's see what it does to the results. 

In [ ]:
# Get the X and y we need
X,y=prep_data(df3)

# We'll use SMOTE_borderline 2 to resample our data, and feed it into the KNN algorithm
sm = SMOTE(kind='borderline2')
X_resampled, y_resampled = sm.fit_sample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled,y_resampled, test_size=0.7, random_state=0)

K=1
clf = neigh.KNeighborsClassifier(K)
clf.fit(X_train, y_train)

predictions1 = clf.predict(X_test)

accuracy = metrics.accuracy_score(y_test, predictions1)*100
print('Accuracy: ' + repr(accuracy) + '%')
print (classification_report(y_test, predictions1))

**Exercise:** What do you think of these results? Is this better/worse than random over-sampling and if so, why?

## Random Undersampling (RUS)

I'm now going to show you how to do random undersampling when dealing with imbalanced classes. As the name suggests, it means taking a sub-sample of the majority class to make it roughly equal to the size of the majority class. You'll see why and when this is pretty much a bad idea, especially when dealing with small datasets. 

This time, we'll go straight in for the undersampling and plotting of the original data and the undersampled data

In [ ]:
X,y=prep_data(df3)

# NOT SO MAGIC STUFF HAPPENING
# Apply random undersampling
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_sample(X, y)

# Two subplots in a figure
f, (ax1, ax2) = plt.subplots(1, 2)

# Plot 1, normal data
c0 = ax1.scatter(X[y == 0, 0], X[y == 0, 1], label="Class #0",alpha=0.5)
c1 = ax1.scatter(X[y == 1, 0], X[y == 1, 1], label="Class #1",alpha=0.5)
ax1.set_title('Original set')

# Plot 2, undersampled data
ax2.scatter(X_resampled[y_resampled == 0, 0], X_resampled[y_resampled == 0, 1], label="Class #0", alpha=.5)
ax2.scatter(X_resampled[y_resampled == 1, 0], X_resampled[y_resampled == 1, 1],label="Class #1", alpha=.5)
ax2.set_title('Random over-sampling')

plt.figlegend((c0, c1), ('Class #0', 'Class #1'), loc='lower center',
              ncol=2, labelspacing=0.)
plt.tight_layout(pad=3)
plt.show()

OK I think the picture says it all, why on earth would you want to throw out data? Especially when your dataset is relatively small, or when your minority class is very small (think about fraud cases) this is NOT a good idea. The under-sampled majority class can look quite different from the actual class, so you don't want to introduce bias by doing this. Only when you have a very very large dataset, and it's computationally heavy to oversample and create more data, should this technique become relevant. 

## Alternative classification metrics

The imblearn library has a bunch of neat, additional evaluation metrics you can use for your imbalanced data. To learn more have a look on their website: 

http://contrib.scikit-learn.org/imbalanced-learn/auto_examples/index.html#evaluation-examples

I'll show you how to create a larger classification report, and how to create a few other metrics

You might notice I go straight from imbalanced data, to using SMOTE and KNN in one line, I use a thing called pipeline. but you can read on how to do that here: http://contrib.scikit-learn.org/imbalanced-learn/api.html#module-imblearn.pipeline

In [ ]:
from imblearn.metrics import (geometric_mean_score,
                              make_index_balanced_accuracy)

X,y=prep_data(df3)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.7, random_state=0)

pipeline = pl.make_pipeline(SMOTE(random_state=0),
                            neigh.KNeighborsClassifier(1))

# Train the classifier with balancing
pipeline.fit(X_train, y_train)

# Test the classifier and get the prediction
y_pred_bal = pipeline.predict(X_test)

# Show the classification report
print(classification_report_imbalanced(y_test, y_pred_bal))

The geometric mean (G-mean) is the root of the product of class-wise sensitivity. This measure tries to maximize the accuracy on each of the classes while keeping these accuracies balanced. For binary classification G-mean is the squared root of the product of the sensitivity and specificity. For multi-class problems it is a higher root of the product of sensitivity for each class.

The best value is 1 and the worst value is 0. Traditionally if at least one class is unrecognized by the classifier, G-mean resolves to zero. Find more here: http://contrib.scikit-learn.org/imbalanced-learn/generated/imblearn.metrics.geometric_mean_score.html

In [ ]:
print('The geometric mean is {}'.format(geometric_mean_score(
    y_test,
    y_pred_bal)))

The index balanced accuracy can transform any metric to be used in imbalanced learning problems. It can be used for evaluating learning processes in two-class imbalanced domains only. The method combines an unbiased index of its overall accuracy and a measure about how dominant is the class with the highest individual accuracy rate (alpha).

The paper that describes this metric is: https://link.springer.com/chapter/10.1007/978-3-642-02172-5_57

In [ ]:
alpha = 0.1
geo_mean = make_index_balanced_accuracy(alpha=alpha, squared=True)(
    geometric_mean_score)

print('The IBA using alpha = {} and the geometric mean: {}'.format(
    alpha, geo_mean(
        y_test,
        y_pred_bal)))

alpha = 0.5
geo_mean = make_index_balanced_accuracy(alpha=alpha, squared=True)(
    geometric_mean_score)

print('The IBA using alpha = {} and the geometric mean: {}'.format(
    alpha, geo_mean(
        y_test,
        y_pred_bal)))

Copyright © ASI 2017 All rights reserved